In [ ]:
import pandas as pd
import numpy as np

def calculateMeanAndMedianValue(sampleName,targets,cqValue):
    cqValuesForMeanAndMedian = zip(sampleName,targets,cqValue)
    meanTableDf = pd.DataFrame(cqValuesForMeanAndMedian, columns=['SampleName', 'Target','CqValue'])
    groupDataBySampleAndTarget = meanTableDf.groupby(['SampleName', 'Target'],as_index=False, sort = False)
    meanTableDf['CqValue']= meanTableDf['CqValue'].replace("Undetermined",np.NaN)
    
    cqMeanTable = groupDataBySampleAndTarget.aggregate(np.mean) 
    cqMeanTable['CqValue'] = cqMeanTable['CqValue'].round(2)
    cqmeanTableDf=cqMeanTable.pivot(index='Target', columns='SampleName', values='CqValue')

    cqMedianTable = groupDataBySampleAndTarget.aggregate(np.median) 
    cqMedianTable['CqValue'] = cqMedianTable['CqValue'].round(2)
    cqMedianTableDf=cqMedianTable.pivot(index='Target', columns='SampleName', values='CqValue')

    return cqmeanTableDf, cqMedianTableDf
    

In [ ]:
def getClinicalCallCounts(sampleName, clinicalCallCounts):
    uniqueSamples = np.unique(sampleName)
    callCount = []
    uniqueCalls = []
    mergedSamples=[]
    for clinicalCallCount in clinicalCallCounts:
        for samples in uniqueSamples:
            uniqueCalls.append(clinicalCallCount)
            mergedSamples.append(samples)
            callCount.append(clinicalCallCounts.get(clinicalCallCount).count(samples))

    listCounts = zip(mergedSamples, uniqueCalls,callCount)
    
    dataCountTable = pd.DataFrame(listCounts, columns=['SampleName', 'Clinical_Call','Counts'])
    clinicalCallTable=dataCountTable.pivot(index='Clinical_Call',columns='SampleName',values='Counts')
    return clinicalCallTable

In [ ]:
def getClinicalCalls(clinicalCall, reporterDyesName, cqValues, repIdentifier):
    bigDF = pd.DataFrame()
    clinicalCallCounts = dict()
    for key in clinicalCall:
        listCqValues = []
        for dyeName in reporterDyesName:
            matchDyeNames = 0
            for cqValue in cqValues.get(key):
                dyeNameAndCq = cqValue.split("\t")
                if dyeNameAndCq[0] == dyeName:
                    listCqValues.append(dyeNameAndCq[1])
                    matchDyeNames=1
            if matchDyeNames == 0:
                listCqValues.append("")
        
        if any( clinicalCall.get(key)):
            calls = list(filter(None, clinicalCall.get(key)))
            call = "|".join(calls)
        else:
            call = "NEG"

        d = key.split("\t")
        listL=[d[0],d[1],d[2]]
        listL.extend(listCqValues)
        listL.append(call)
        samplesNames = d[2].split(repIdentifier)
        clinicalCallCounts.setdefault(call,[]).append(samplesNames[0])
        dataFrame = pd.DataFrame([listL],[call])
        bigDF = pd.concat([bigDF,dataFrame])
        clincalCallsForEachWell = pd.DataFrame(bigDF.values[0:],columns= header)
   
    return clincalCallsForEachWell, clinicalCallCounts

In [ ]:
repIdentifier = "-"
inFolder = "D:/Covid_data_New/Yr2023/5May/18May/"
inFile = inFolder+"Plate 160 True Mark  Psyche 19 Apr 2023_20230516 125556.xlsx"
readSepsisFile = pd.read_excel(inFile,sheet_name="Results",skiprows=23)

reporterDyesName = np.unique(readSepsisFile["Reporter"])
header = ['Well','well Position','Sample']
header.extend(reporterDyesName)
header.append('Call')

targets = readSepsisFile["Target"]
sample = readSepsisFile["Sample"]

sampleName = []
for samples in sample:
    ss = samples.split("-")
    sampleName.append(ss[0])
cqValue = readSepsisFile["Cq"]

cqMeanTableDf, cqMedianTable = calculateMeanAndMedianValue(sampleName,targets,cqValue)


In [ ]:

clinicalCall = dict()
cqValues = dict()

for i in readSepsisFile.index:
    well = readSepsisFile.loc[i,"Well"]
    wellPosition = readSepsisFile.loc[i,"Well Position"]
    Cq = readSepsisFile.loc[i,"Cq"]
    reporterDye = readSepsisFile.loc[i,"Reporter"]
    targetSample = readSepsisFile.loc[i,"Target"]
    keyForClinicalCall = str(well)+"\t"+str(wellPosition)+"\t"+str(readSepsisFile.loc[i,"Sample"])
    if Cq == "Undetermined":
        clinicalCall.setdefault(keyForClinicalCall,[]).append("")
        cqValues.setdefault(keyForClinicalCall,[]).append(reporterDye+"\t"+str(Cq))
        continue
    clinicalCall.setdefault(keyForClinicalCall,[]).append(targetSample)
    cqValues.setdefault(keyForClinicalCall,[]).append(reporterDye+"\t"+str(Cq))


In [ ]:


clincalCallsForEachWell, clinicalCallCounts = getClinicalCalls(clinicalCall, reporterDyesName, cqValues, repIdentifier)
clinicalCallTable = getClinicalCallCounts(sampleName, clinicalCallCounts)

excelWriter = pd.ExcelWriter(inFolder+"/aa_new.xlsx")
clincalCallsForEachWell.to_excel(excel_writer=excelWriter,sheet_name= "ClinicalCall",index=False)
cqMeanTableDf.to_excel(excel_writer=excelWriter,sheet_name= "Mean")
cqMedianTable.to_excel(excel_writer=excelWriter,sheet_name= "Median")
clinicalCallTable.to_excel(excel_writer=excelWriter,sheet_name= "ClinicalCallCount")
excelWriter.close()